In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "/Users/apple/quant_course/week1/week1_3/factor/LiangWen"
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df

,code,Turn20,PctTurn20,STR,date
0,000001.XSHE,NaN,NaN,NaN,2020-01-02
1,000001.XSHE,NaN,NaN,NaN,2020-01-03
2,000001.XSHE,NaN,NaN,NaN,2020-01-06
3,000001.XSHE,NaN,NaN,NaN,2020-01-07
4,000001.XSHE,NaN,NaN,NaN,2020-01-08
...,...,...,...,...,...
4253855,689009.XSHG,0.838920,0.432636,0.460271,2023-11-24
4253856,689009.XSHG,0.871365,0.498172,0.462728,2023-11-27
4253857,689009.XSHG,0.907240,0.570637,0.450450,2023-11-28
4253858,689009.XSHG,0.944350,0.652413,0.444356,2023-11-29


In [2]:
df = result_df.copy()
df['volume_std'] = df.groupby('code')['volume'].transform(lambda x:x.rolling(window=20).std())
df['volume_mean'] = df.groupby('code')['volume'].transform(lambda x:x.rolling(window=20).mean())
df['Z'] = df['volume_std'] / df['volume_mean']
df['ret'] = df['close'] / df.groupby('code')['close'].shift(1) - 1
df['rank'] = df.groupby('code')['Z'].transform(lambda x: x.rolling(window=20).rank(pct=True))
df

,date,code,open,close,low,high,volume,money,turnover_ratio,volume_std,volume_mean,Z,ret,rank
0,2020-01-02,000001.XSHE,1976.78,2002.90,1964.91,2012.40,1288881.0,2.571196e+09,0.7885,NaN,NaN,NaN,NaN,NaN
1,2020-01-03,000001.XSHE,2011.21,2039.71,2008.84,2055.14,940146.0,1.914495e+09,0.5752,NaN,NaN,NaN,0.018378,NaN
2,2020-01-06,000001.XSHE,2019.52,2026.65,2007.65,2058.70,726114.0,1.477930e+09,0.4442,NaN,NaN,NaN,-0.006403,NaN
3,2020-01-07,000001.XSHE,2033.77,2036.14,2012.40,2051.58,613690.0,1.247047e+09,0.3755,NaN,NaN,NaN,0.004683,NaN
4,2020-01-08,000001.XSHE,2018.34,1977.97,1974.41,2024.27,714104.0,1.423609e+09,0.4369,NaN,NaN,NaN,-0.028569,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4253855,2023-11-24,689009.XSHG,34.88,35.28,34.88,35.88,10815571.0,3.834648e+08,2.0661,2.409382e+06,4391589.15,0.548636,0.020538,1.00
4253856,2023-11-27,689009.XSHG,35.24,34.89,34.62,35.60,6348515.0,2.225542e+08,1.2128,2.422234e+06,4561422.65,0.531026,-0.011054,0.85
4253857,2023-11-28,689009.XSHG,34.90,34.36,34.01,35.17,5755962.0,1.979950e+08,1.0996,2.357968e+06,4749198.95,0.496498,-0.015191,0.60
4253858,2023-11-29,689009.XSHG,34.59,33.27,33.09,34.59,6423819.0,2.155980e+08,1.2271,2.326078e+06,4943469.95,0.470535,-0.031723,0.40


In [3]:
result_df['Turn20'] = result_df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).mean())
result_df

,date,code,open,close,low,high,volume,money,turnover_ratio,Turn20
0,2020-01-02,000001.XSHE,1976.78,2002.90,1964.91,2012.40,1288881.0,2.571196e+09,0.7885,NaN
1,2020-01-03,000001.XSHE,2011.21,2039.71,2008.84,2055.14,940146.0,1.914495e+09,0.5752,NaN
2,2020-01-06,000001.XSHE,2019.52,2026.65,2007.65,2058.70,726114.0,1.477930e+09,0.4442,NaN
3,2020-01-07,000001.XSHE,2033.77,2036.14,2012.40,2051.58,613690.0,1.247047e+09,0.3755,NaN
4,2020-01-08,000001.XSHE,2018.34,1977.97,1974.41,2024.27,714104.0,1.423609e+09,0.4369,NaN
...,...,...,...,...,...,...,...,...,...,...
4253855,2023-11-24,689009.XSHG,34.88,35.28,34.88,35.88,10815571.0,3.834648e+08,2.0661,0.838920
4253856,2023-11-27,689009.XSHG,35.24,34.89,34.62,35.60,6348515.0,2.225542e+08,1.2128,0.871365
4253857,2023-11-28,689009.XSHG,34.90,34.36,34.01,35.17,5755962.0,1.979950e+08,1.0996,0.907240
4253858,2023-11-29,689009.XSHG,34.59,33.27,33.09,34.59,6423819.0,2.155980e+08,1.2271,0.944350


In [4]:
df = result_df.copy()
df['tmp1'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=60).sum())
df['tmp2'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).sum())
df['tmp'] = (df['tmp1'] - df['tmp2']) / 40
df['tmp'] = df['turnover_ratio'] / df['tmp'] - 1
result_df['PctTurn20'] = df.groupby('code')['tmp'].transform(lambda x: x.rolling(window=20).mean())

In [5]:
df = result_df.copy()
result_df['STR'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).std())

In [6]:
output_df = result_df[['code','date','Turn20','PctTurn20','STR']]
output_df.sort_values(by=['date','code'],ascending=True)
output_df

,code,date,Turn20,PctTurn20,STR
0,000001.XSHE,2020-01-02,NaN,NaN,NaN
1,000001.XSHE,2020-01-03,NaN,NaN,NaN
2,000001.XSHE,2020-01-06,NaN,NaN,NaN
3,000001.XSHE,2020-01-07,NaN,NaN,NaN
4,000001.XSHE,2020-01-08,NaN,NaN,NaN
...,...,...,...,...,...
4253855,689009.XSHG,2023-11-24,0.838920,0.432636,0.460271
4253856,689009.XSHG,2023-11-27,0.871365,0.498172,0.462728
4253857,689009.XSHG,2023-11-28,0.907240,0.570637,0.450450
4253858,689009.XSHG,2023-11-29,0.944350,0.652413,0.444356


In [57]:
output_folder = '/Users/apple/Desktop/week1_1.1/week1_3/new_feature/LiangWen'
select_columns = ['code', 'Turn20','PctTurn20','STR']
os.makedirs(output_folder,exist_ok=True)
#result_df['date']=pd.to_datetime(result_df['date'])

date_index = pd.date_range(result_df['date'].min(), result_df['date'].max(),freq='D')
exist_date = [d for d in date_index if str(d)[:10] in result_df['date'].values]



In [58]:
#date_index
#result_df['date'].values[1] == str(date_index[1])[:10]
exist_date

[Timestamp('2020-01-02 00:00:00'),
 Timestamp('2020-01-03 00:00:00'),
 Timestamp('2020-01-06 00:00:00'),
 Timestamp('2020-01-07 00:00:00'),
 Timestamp('2020-01-08 00:00:00'),
 Timestamp('2020-01-09 00:00:00'),
 Timestamp('2020-01-10 00:00:00'),
 Timestamp('2020-01-13 00:00:00'),
 Timestamp('2020-01-14 00:00:00'),
 Timestamp('2020-01-15 00:00:00'),
 Timestamp('2020-01-16 00:00:00'),
 Timestamp('2020-01-17 00:00:00'),
 Timestamp('2020-01-20 00:00:00'),
 Timestamp('2020-01-21 00:00:00'),
 Timestamp('2020-01-22 00:00:00'),
 Timestamp('2020-01-23 00:00:00'),
 Timestamp('2020-02-03 00:00:00'),
 Timestamp('2020-02-04 00:00:00'),
 Timestamp('2020-02-05 00:00:00'),
 Timestamp('2020-02-06 00:00:00'),
 Timestamp('2020-02-07 00:00:00'),
 Timestamp('2020-02-10 00:00:00'),
 Timestamp('2020-02-11 00:00:00'),
 Timestamp('2020-02-12 00:00:00'),
 Timestamp('2020-02-13 00:00:00'),
 Timestamp('2020-02-14 00:00:00'),
 Timestamp('2020-02-17 00:00:00'),
 Timestamp('2020-02-18 00:00:00'),
 Timestamp('2020-02-

In [62]:

for date in exist_date:
    tmp = result_df.loc[result_df['date'] == str(date)[:10], select_columns]
    tmp = tmp.sort_values(by='code',ascending=True)
    file_name = os.path.join(output_folder, f'{str(date)[:10]}.csv')
    tmp.to_csv(file_name, index=False)

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "/Users/apple/quant_course/week1/week1_3/factor/LiangWen"
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df

,code,Turn20,PctTurn20,STR,date
0,000001.XSHE,NaN,NaN,NaN,2020-01-02
1,000001.XSHE,NaN,NaN,NaN,2020-01-03
2,000001.XSHE,NaN,NaN,NaN,2020-01-06
3,000001.XSHE,NaN,NaN,NaN,2020-01-07
4,000001.XSHE,NaN,NaN,NaN,2020-01-08
...,...,...,...,...,...
4253855,689009.XSHG,0.838920,0.432636,0.460271,2023-11-24
4253856,689009.XSHG,0.871365,0.498172,0.462728,2023-11-27
4253857,689009.XSHG,0.907240,0.570637,0.450450,2023-11-28
4253858,689009.XSHG,0.944350,0.652413,0.444356,2023-11-29


In [3]:
df = result_df.copy()
df['score1'] = df.groupby('date')['STR'].transform(lambda x: x.rank())
df['tag1'] = df.groupby('date')['STR'].transform(lambda x: x.rank(pct=True))
df['score2'] = 0
tmp1 = df[df['tag1'] < 0.5]
tmp1['score2'] = tmp1.groupby('date')['Turn20'].transform(lambda x: x.rank(ascending=False))
tmp2 = df[df['tag1'] >= 0.5]
tmp2['score2'] = tmp2.groupby('date')['Turn20'].transform(lambda x: x.rank())
final = pd.concat([tmp1, tmp2]).sort_values(by=['date', 'code'])
final = final.reset_index(drop=True)
final['UTR'] = (final['score1'] + final['score2'])


In [4]:
final

,code,Turn20,PctTurn20,STR,date,score1,tag1,score2,UTR
0,000001.XSHE,0.553360,NaN,0.214226,2020-02-06,314.0,0.084545,1492.0,1806.0
1,000002.XSHE,0.813605,NaN,0.263091,2020-02-06,455.0,0.122509,1184.0,1639.0
2,000004.XSHE,2.422650,NaN,0.953548,2020-02-06,1873.0,0.504308,367.0,2240.0
3,000005.XSHE,0.984995,NaN,0.409240,2020-02-06,861.0,0.231826,975.0,1836.0
4,000006.XSHE,0.761105,NaN,0.235302,2020-02-06,373.0,0.100431,1253.0,1626.0
...,...,...,...,...,...,...,...,...,...
4155038,688799.XSHG,2.267325,0.385494,0.859976,2023-11-30,2966.0,0.584664,729.0,3695.0
4155039,688800.XSHG,2.292970,0.306996,1.545284,2023-11-30,3641.0,0.717721,749.0,4390.0
4155040,688819.XSHG,1.315320,0.300737,0.379709,2023-11-30,1795.0,0.353834,476.0,2271.0
4155041,688981.XSHG,1.173325,-0.304892,0.525994,2023-11-30,2285.0,0.450424,647.0,2932.0


In [5]:
#将日期列转换为datetime类型
df = final
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='/Users/apple/quant_course/week1/week1_3/factor/UTR'
selected_columns=['code', 'UTR']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [6]:
final

,code,Turn20,PctTurn20,STR,date,score1,tag1,score2,UTR
0,000001.XSHE,0.553360,NaN,0.214226,2020-02-06,314.0,0.084545,1492.0,1806.0
1,000002.XSHE,0.813605,NaN,0.263091,2020-02-06,455.0,0.122509,1184.0,1639.0
2,000004.XSHE,2.422650,NaN,0.953548,2020-02-06,1873.0,0.504308,367.0,2240.0
3,000005.XSHE,0.984995,NaN,0.409240,2020-02-06,861.0,0.231826,975.0,1836.0
4,000006.XSHE,0.761105,NaN,0.235302,2020-02-06,373.0,0.100431,1253.0,1626.0
...,...,...,...,...,...,...,...,...,...
4155038,688799.XSHG,2.267325,0.385494,0.859976,2023-11-30,2966.0,0.584664,729.0,3695.0
4155039,688800.XSHG,2.292970,0.306996,1.545284,2023-11-30,3641.0,0.717721,749.0,4390.0
4155040,688819.XSHG,1.315320,0.300737,0.379709,2023-11-30,1795.0,0.353834,476.0,2271.0
4155041,688981.XSHG,1.173325,-0.304892,0.525994,2023-11-30,2285.0,0.450424,647.0,2932.0


In [7]:
import statsmodels.api as sm
results = sm.OLS(final['UTR'], final[['STR', 'Turn20']]).fit()    
final['UTR_resid']=results.resid    #取残差项作为中性化后的因子值
final

,code,Turn20,PctTurn20,STR,date,score1,tag1,score2,UTR,UTR_resid
0,000001.XSHE,0.553360,NaN,0.214226,2020-02-06,314.0,0.084545,1492.0,1806.0,1470.322576
1,000002.XSHE,0.813605,NaN,0.263091,2020-02-06,455.0,0.122509,1184.0,1639.0,1166.684298
2,000004.XSHE,2.422650,NaN,0.953548,2020-02-06,1873.0,0.504308,367.0,2240.0,763.974719
3,000005.XSHE,0.984995,NaN,0.409240,2020-02-06,861.0,0.231826,975.0,1836.0,1227.063658
4,000006.XSHE,0.761105,NaN,0.235302,2020-02-06,373.0,0.100431,1253.0,1626.0,1188.586333
...,...,...,...,...,...,...,...,...,...,...
4155038,688799.XSHG,2.267325,0.385494,0.859976,2023-11-30,2966.0,0.584664,729.0,3695.0,2326.881034
4155039,688800.XSHG,2.292970,0.306996,1.545284,2023-11-30,3641.0,0.717721,749.0,4390.0,2729.964262
4155040,688819.XSHG,1.315320,0.300737,0.379709,2023-11-30,1795.0,0.353834,476.0,2271.0,1526.094542
4155041,688981.XSHG,1.173325,-0.304892,0.525994,2023-11-30,2285.0,0.450424,647.0,2932.0,2190.878540


In [10]:
def softsign(x):
    return x/(1+abs(x))

final['UTR2.0'] = final['STR'] + final['STR'].transform(lambda x: softsign(x)) * final['Turn20']
final

,code,Turn20,PctTurn20,STR,date,score1,tag1,score2,UTR,UTR_resid,UTR2.0
0,000001.XSHE,0.553360,NaN,0.214226,2020-02-06,314.0,0.084545,1492.0,1806.0,1470.322576,0.311855
1,000002.XSHE,0.813605,NaN,0.263091,2020-02-06,455.0,0.122509,1184.0,1639.0,1166.684298,0.432558
2,000004.XSHE,2.422650,NaN,0.953548,2020-02-06,1873.0,0.504308,367.0,2240.0,763.974719,2.136070
3,000005.XSHE,0.984995,NaN,0.409240,2020-02-06,861.0,0.231826,975.0,1836.0,1227.063658,0.695281
4,000006.XSHE,0.761105,NaN,0.235302,2020-02-06,373.0,0.100431,1253.0,1626.0,1188.586333,0.380278
...,...,...,...,...,...,...,...,...,...,...,...
4155038,688799.XSHG,2.267325,0.385494,0.859976,2023-11-30,2966.0,0.584664,729.0,3695.0,2326.881034,1.908293
4155039,688800.XSHG,2.292970,0.306996,1.545284,2023-11-30,3641.0,0.717721,749.0,4390.0,2729.964262,2.937384
4155040,688819.XSHG,1.315320,0.300737,0.379709,2023-11-30,1795.0,0.353834,476.0,2271.0,1526.094542,0.741698
4155041,688981.XSHG,1.173325,-0.304892,0.525994,2023-11-30,2285.0,0.450424,647.0,2932.0,2190.878540,0.930427


In [11]:
def tanh(x):
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

final['UTR2.0_tanh'] = final['STR'] + final['STR'].transform(lambda x: tanh(x)) * final['Turn20']


In [12]:
import statsmodels.api as sm
result = sm.OLS(final['UTR2.0'],final[['Turn20','STR']]).fit()
final['UTR2.0_resd'] = result.resid
result = sm.OLS(final['UTR2.0_tanh'],final[['Turn20','STR']]).fit()
final['UTR2.0_tanh_resd'] = result.resid

In [13]:
final

,code,Turn20,PctTurn20,STR,date,score1,tag1,score2,UTR,UTR_resid,UTR2.0,UTR2.0_tanh,UTR2.0_resd,UTR2.0_tanh_resd
0,000001.XSHE,0.553360,NaN,0.214226,2020-02-06,314.0,0.084545,1492.0,1806.0,1470.322576,0.311855,0.330989,-0.335144,-0.410794
1,000002.XSHE,0.813605,NaN,0.263091,2020-02-06,455.0,0.122509,1184.0,1639.0,1166.684298,0.432558,0.472338,-0.455873,-0.560627
2,000004.XSHE,2.422650,NaN,0.953548,2020-02-06,1873.0,0.504308,367.0,2240.0,763.974719,2.136070,2.749665,-0.715497,-0.515315
3,000005.XSHE,0.984995,NaN,0.409240,2020-02-06,861.0,0.231826,975.0,1836.0,1227.063658,0.695281,0.791248,-0.490205,-0.560175
4,000006.XSHE,0.761105,NaN,0.235302,2020-02-06,373.0,0.100431,1253.0,1626.0,1188.586333,0.380278,0.411157,-0.437727,-0.543133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155038,688799.XSHG,2.267325,0.385494,0.859976,2023-11-30,2966.0,0.584664,729.0,3695.0,2326.881034,1.908293,2.438590,-0.721157,-0.581002
4155039,688800.XSHG,2.292970,0.306996,1.545284,2023-11-30,3641.0,0.717721,749.0,4390.0,2729.964262,2.937384,3.638774,-0.540165,-0.165987
4155040,688819.XSHG,1.315320,0.300737,0.379709,2023-11-30,1795.0,0.353834,476.0,2271.0,1526.094542,0.741698,0.856454,-0.639331,-0.762784
4155041,688981.XSHG,1.173325,-0.304892,0.525994,2023-11-30,2285.0,0.450424,647.0,2932.0,2190.878540,0.930427,1.091905,-0.528368,-0.560716


In [14]:
#将日期列转换为datetime类型
df = final
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='/Users/apple/quant_course/week1/week1_3/factor/UTR2.0'
selected_columns=['code', 'UTR2.0','UTR2.0_resd','UTR2.0_tanh','UTR2.0_tanh_resd']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)